In [7]:
import anndata as ad
import episcanpy.api as epi
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sns
import time

import episcanpy
print(episcanpy.__version__)

0.1.7+7.g9c12d96


# Make the count matrix

In [13]:
# download the data from 10x website
!wget http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_fragments.tsv.gz
!wget http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_fragments.tsv.gz.tbi
!wget http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_singlecell.csv
!wget http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_peaks.bed

--2019-12-04 13:58:40--  http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_fragments.tsv.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 52.222.158.122, 52.222.158.126, 52.222.158.176, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|52.222.158.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978599242 (933M) [text/tab-separated-values]
Saving to: 'atac_v1_pbmc_5k_fragments.tsv.gz'

atac_v1_pbmc_5k_fra 100%[===================>] 933.26M  11.2MB/s    in 76s     

2019-12-04 13:59:57 (12.3 MB/s) - 'atac_v1_pbmc_5k_fragments.tsv.gz' saved [978599242/978599242]

--2019-12-04 13:59:57--  http://cf.10xgenomics.com/samples/cell-atac/1.0.1/atac_v1_pbmc_5k/atac_v1_pbmc_5k_fragments.tsv.gz.tbi
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 143.204.214.26, 143.204.214.27, 143.204.214.38, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|143.204.214.26|:80... connected.
HTTP request sent, awaiting response..

In [3]:
DATADIR = ''
peak_file = 'atac_v1_pbmc_5k_peaks.bed' # annotation 
tsv_file = 'atac_v1_pbmc_5k_fragments.tsv.gz' # fragment file
csv_file = 'atac_v1_pbmc_5k_singlecell.csv'  # to filter bad barcodes according to cellranger
output_mtx = 'atac_v1_pbmc_5k_singlecell_peaks.h5ad'

In [4]:
start = time.time()

# loading the peaks
peaks = []
with open(DATADIR+peak_file) as f:
    for line in f:
        line = line.rstrip('\n').split('\t')
        peaks.append([line[0], int(line[1]), int(line[2])])
        
# build the count matrix with all barcodes passing cellranger quality control
adata = epi.ct.bld_mtx_fly(tsv_file=tsv_file,
            annotation=peaks,
            csv_file=csv_file,
            genome=None,
            DATADIR=DATADIR, 
            save=output_mtx)

end = time.time()
print(end - start)
print((end - start)/60)

loading and filtering the barcodes
building the count matrix
making the AnnData
11750.689690113068
195.84482816855112


In [6]:
annotation = '../../input/metadata.tsv'
df= pd.read_csv(annotation, sep='\t')

##############
annot_barcode = df.iloc[:,0].tolist()
annot_label = df.iloc[:,1].tolist()
#############
cell_barcodes = adata.obs_names.tolist()
#############
cell_label = []

for cell in cell_barcodes:
    if cell not in annot_barcode:
        print(cell)
        break
    index = 0
    for cell2 in annot_barcode:
        if cell == cell2:
            cell_label.append(annot_label[index])
            break
        index += 1
#############
adata.obs['cell_label'] = [str(x) for x in cell_label]
adata

AnnData object with n_obs × n_vars = 5335 × 97998 
    obs: 'cell_label'

# Downstream Analysis